In [10]:
import os
import pickle
import re

import pandas as pd
import pdfplumber
from sqlalchemy import create_engine

from src.table_extraction_tools.crosstab_normalization import normalize_crosstab

# Import environment variables
from dotenv import load_dotenv
load_dotenv()

# Define output directory
output_dir = os.getenv("OUTPUT_DIR")

# Open PDF file to extract tables (load path from dotenv)
pdf_path = os.getenv("PDF_PATH")
pdf = pdfplumber.open(pdf_path)

# Define pages to skip
skip_pages = [1, 2, 69, 100, 111, 122, 125, 134, 156, 167, 170, 174, 201, 228, 238, 269, 271, 273, 282]

# Configuration lookup for each table
table_config = {
    "A-1": 1, # 3
    "A-2": 1, # 70
    "A-3": 2, # 101
    "A-4": 2, # 112
    "A-5": 1, # 123
    "A-6": 2, # 126
    "A-7": 1, # 135
    "A-8": 1, # 157
    "A-9": 3, # 168
    "A-10": 2, # 171
    "A-11": 4, # 175
    "A-12": [4,5], # 202
    "A-13": [4,6,6], # 229
    "A-14": 4, # 239
    "A-15": 7, # 270
    "A-16": 3, # 272
    "A-17": 2, # 274
    "A-18": 8 # 283
}

# Define the settings for normalizing the table
normalization_settings = {
    1: {"header_row_index":5,"id_columns":4,"data_column_names":4},
    2: {"header_row_index":4,"id_columns":4,"data_column_names":4},
    3: {"header_row_index":5,"id_columns":2,"data_column_names":1},
    4: {"header_row_index":5,"id_columns":5,"data_column_names":4},
    5: {"header_row_index":4,"id_columns":5,"data_column_names":4},
    6: {"header_row_index":0,"id_columns":5,"data_column_names":4},
    7: {"header_row_index":4,"id_columns":2,"data_column_names":1},
    8: {"header_row_index":4,"id_columns":1,"data_column_names":1}
}

def clean_names(names):
    # Remove trailing leading and extra whitespace, convert to lowercase, replace spaces and hyphens with underscores, and remove all other characters
    names = names.str.strip()
    names = names.str.lower()
    names = names.str.replace(r"[\/_-]+", " ", regex=True)
    names = names.str.replace(r"[^a-z\s]", "", regex=True)
    names = names.str.strip()
    names = names.str.replace(r"[\s]+", "_", regex=True)
    
    return names

In [5]:
# Initialize an empty list
all_tables = []
table_numbers = []
table_count = 0

# Initialize the last table number, headers, 
last_table_number = ""
last_table_headers = []
last_table_config = {}

# Loop through each page in the PDF
for page_num in range(1, 292):
    # Skip pages that don't contain tables
    if page_num in skip_pages:
        print(f"Skipping page {page_num}...")
        continue
    
    # Extract the page
    p0 = pdf.pages[page_num-1]
    # im = p0.to_image(resolution=150)
    
    # Extract all text from the page and print it
    text = p0.extract_text()

    # Extract table title, sub-title, and site name as first three lines of text
    title = text.split("\n")[0]
    subtitle = text.split("\n")[1]
    site_name = text.split("\n")[2]
    # print([title, subtitle, site_name])
    
    # Get table number from title and set normalization settings
    table_number = re.findall(r'^[A-Z]+\s([A-Z]-\d+)', title)[0]
    if table_number == last_table_number:
        table_page_count = table_page_count + 1
    else:
        table_numbers = table_numbers + [table_number]
        table_page_count = 1
        table_count = table_count + 1
        all_tables.append(pd.DataFrame())
    
    # Print the table number and page number
    print(f"Extracting table {table_number} part {table_page_count} from page {page_num}...")
    
    # Set normalization settings
    table_config_number = table_config[table_number]
    
    # If table_config_number is an integer, use that as the index for normalization_settings
    if isinstance(table_config_number, int):
        normalize_settings = normalization_settings[table_config_number].copy()
    # If table_config_number is a list, use the modulous of the table_page_count to get the index for normalization_settings based on the length of the list
    # (e.g. if table_config_number is [5,6] and table_page_count is 1, use 5; if table_page_count is 2, use 6; if table_page_count is 3, use 5; etc.)
    elif isinstance(table_config_number, list):
        num_config = len(table_config_number)
        config_number = table_config_number[(table_page_count-1) % num_config]
        normalize_settings = normalization_settings[config_number].copy()
    
    # Get the header row index from the normalization settings
    n = normalize_settings["header_row_index"]
    m = normalize_settings["id_columns"]
    
    # Define table settings to find the upper table boundaries
    if (table_number == "A-14"):
        table_settings={
            "vertical_strategy": "lines",
            "horizontal_strategy": "lines",
        }
    else:
        table_settings={
            "vertical_strategy": "lines",
            "horizontal_strategy": "text"
        }

    # Find the table, extract title and headers, and display them
    tables = p0.find_tables(table_settings)
    
    # Loop through each table on the page
    for table in tables:
        # Obtain the bounding box of the table and crop the page to the table
        bounding_box = table.bbox
        bounding_box = (0, bounding_box[1], p0.width, bounding_box[3])
        p1 = p0.crop(bounding_box)
        # im_cropped = p1.to_image()
        
        # Define table settings to find the table headers and vertical lines
        table_settings={
            "vertical_strategy": "lines",
            "horizontal_strategy": "text",
            "intersection_x_tolerance": 15
        }

        # Find the table and remove empty rows
        table_cropped = p1.find_table(table_settings)
        extracted_data = table_cropped.extract()
        # im_cropped.reset().debug_tablefinder(table_settings)

        # Get index of empty rows and remove them from extracted data
        empty_rows = [i for i, row in enumerate(extracted_data) if not any(row)]
        extracted_data = [row for row in extracted_data if any(row)]
        extracted_rows = table_cropped.rows
        extracted_rows = [row for i, row in enumerate(extracted_rows) if i not in empty_rows]
        
        # Remove empty rows and extract headers as first n rows
        extracted_headers = extracted_data[:n]
        
        # Extract the vertical lines from the nth row and add the right boundary of the table
        cell_data = extracted_rows[n].cells
        vertical_lines = [cell[0] for cell in cell_data]
        vertical_lines.append(table_cropped.bbox[2])

        # Define table settings with explicit vertical lines
        table_settings={
            "vertical_strategy": "explicit",
            "explicit_vertical_lines": vertical_lines,
            "horizontal_strategy": "text",
            "intersection_x_tolerance": 15
        }
        
        # Extract table from cropped page with updated table settings and remove empty rows
        extracted_table = p1.extract_table(table_settings)
        extracted_table = [row for row in extracted_table if any(row)]
        # im_cropped.reset().debug_tablefinder(table_settings)

        # Combine text from rows n and n+1 by concatenating the strings vertically and adding a space if table_number not "A-9", "A-15", or "A-16"
        if (table_number not in ["A-9", "A-15", "A-16"]):
            extracted_table[n] = [i + " " + j for i, j in zip(extracted_table[n], extracted_table[n+1])]
            extracted_table.pop(n+1)

        # Remove the first n rows and add headers to the table by concatenating the two dataframes if n > 0
        if (n > 0):
            extracted_table = extracted_table[n:]
            extracted_table = extracted_headers + extracted_table
            last_table_headers = extracted_headers
            last_table_config = normalize_settings.copy()
            
        # If n = 0, add the last table headers to the table and use the last table's header row index and data column headers
        elif (n == 0):
            extracted_table = last_table_headers + extracted_table
            normalize_settings['header_row_index'] = last_table_config['header_row_index']
            n = normalize_settings["header_row_index"]
        
        # If the value in the first row and column is NaN, replace it with "Location ID:" and replace the rest of the row with empty strings
        if pd.isna(extracted_table[0][0]):
            extracted_table[0][0] = "Location ID:"
            extracted_table[0][1:m] = ["" for i in range(1,m)]
        
        # Concatenate the strings vertically and add a space for each column
        concatenated_rows = []
        concatenated_row = ""
        
        # Loop through the cols/rows to concatenate
        for col_index in range(1,m):
            for row_index in range(0,n):
                concatenated_row = concatenated_row + " " + extracted_table[row_index][col_index]
            concatenated_rows.append(concatenated_row)
            concatenated_row = ""
        
        # Remove leading/trailing/extra spaces from the concatenated rows
        concatenated_rows = [row.strip() for row in concatenated_rows]
        
        # Add concatenated rows to the table
        extracted_table[n][1:m] = concatenated_rows
        
        # Add analyte header to the table if empty
        if extracted_table[n][0] in ["", " "]:
            extracted_table[n][0] = "Analyte"
        
        # Convert the table to a Pandas DataFrame and normalize the table
        normalized_df = normalize_crosstab(pd.DataFrame(extracted_table), **normalize_settings)
        
        # Rename column headers
        if table_page_count == 1:
            table_column_names = clean_names(normalized_df.columns)
            normalized_df.columns = table_column_names
        else:
            if (len(normalized_df.columns) == len(all_tables[table_count-1].columns)):
                normalized_df.columns = table_column_names
            else:
                normalized_df.columns = clean_names(normalized_df.columns)
    
        # Add page number, title, subtitle, and site name to the dataframe
        normalized_df["page_number"] = page_num
        normalized_df["table_number"] = table_number
        normalized_df["title"] = title
        normalized_df["subtitle"] = subtitle
        normalized_df["dataset"] = normalized_df['title'].str.replace('SUMMARY OF ANALYTICAL RESULTS FOR ', '')
        normalized_df["site_name"] = site_name
        
        # Concatenate the normalized table to the clean_tables DataFrame
        all_tables[table_count-1] = pd.concat([all_tables[table_count-1], normalized_df])
        
        # Update the last table number
        last_table_number = table_number

# Close the PDF file
pdf.close()

fraw = f"{output_dir}/{pdf_path.split('/')[-1].split('.')[0]}_raw"

# Save the list of tables to a pickle file
with open(f"{fraw}.pkl", "wb") as fout:
    pickle.dump(all_tables, fout)
    
# Save each table to a worksheet in an Excel file using filename from pdf_path
fout = f"{fraw}.xlsx"
with pd.ExcelWriter(fout) as writer:
    for i, table in enumerate(all_tables):
        table.to_excel(writer, sheet_name=table_numbers[i], index=False)


Skipping page 1...
Skipping page 2...
Extracting table A-1 part 1 from page 3...
Extracting table A-1 part 2 from page 4...
Extracting table A-1 part 3 from page 5...
Extracting table A-1 part 4 from page 6...
Extracting table A-1 part 5 from page 7...
Extracting table A-1 part 6 from page 8...
Extracting table A-1 part 7 from page 9...
Extracting table A-1 part 8 from page 10...
Extracting table A-1 part 9 from page 11...
Extracting table A-1 part 10 from page 12...
Extracting table A-1 part 11 from page 13...
Extracting table A-1 part 12 from page 14...
Extracting table A-1 part 13 from page 15...
Extracting table A-1 part 14 from page 16...
Extracting table A-1 part 15 from page 17...
Extracting table A-1 part 16 from page 18...
Extracting table A-1 part 17 from page 19...
Extracting table A-1 part 18 from page 20...
Extracting table A-1 part 19 from page 21...
Extracting table A-1 part 20 from page 22...
Extracting table A-1 part 21 from page 23...
Extracting table A-1 part 22 from

In [35]:
# Read the pickle file from last step
with open(f"{output_dir}/{pdf_path.split('/')[-1].split('.')[0]}_raw.pkl", "rb") as fin:
    all_tables = pickle.load(fin)

# For Table "A-9", split conc_rl column into conc_rl, validator_qualifiers, and mdl using regex and remove conc_rl_qual_mdl column
all_tables[8][["conc_rl", "validator_qualifier", "mdl"]] = all_tables[8]["conc_rl_qual_mdl"].str.extract(r'^([\d.]+)\s*(.+)?\s([\d.]+)$')
all_tables[8] = all_tables[8].drop(columns=["conc_rl_qual_mdl"])

# For Table "A-10", rename "risk_based_action_limt_rsl" to "risk_based_action_limit_rsl"
all_tables[9] = all_tables[9].rename(columns={"risk_based_action_limt_rsl":"risk_based_action_limit_rsl"})

# For Table "A-11", replace analyte "MEtEhTylAbeLnSz e(µnge/L)" with "Ethylbenze"
all_tables[10]["analyte"] = all_tables[10]["analyte"].str.replace("MEtEhTylAbeLnSz e(µnge/L)", "Ethylbenzene")

# For Table "A-15", replace analyte "Asbestos Analysis" with "ASBESTOS (%)"
all_tables[14]["analyte"] = all_tables[14]["analyte"].str.replace("Asbestos Analysis", "ASBESTOS (%)")

# For Table "A-16", combine the blank column with the "client_id" and drop the "" column
all_tables[15]["client_id"] = all_tables[15]["client_id"] + " " + all_tables[15][""]
all_tables[15]["client_id"] = all_tables[15]["client_id"].str.strip()
all_tables[15] = all_tables[15].drop(columns=[""])

# For Table "A-16", replace analyte "Lead" with "Lead (mg/kg)" and replace analyte "Concentration" with "Lead"
all_tables[15]["analyte"] = all_tables[15]["analyte"].str.replace("Lead", "METALS (mg/kg)")
all_tables[15]["analyte"] = all_tables[15]["analyte"].str.replace("Concentration", "Lead")

# For Table "A-18", replace analyte "(ng/g)" with "PCB CONGENERS (ng/g)"
all_tables[17]["analyte"] = all_tables[17]["analyte"].str.replace("(ng/g)", "PCB CONGENERS (ng/g)")

# Concatenate the list of tables into a single DataFrame
clean_tables = pd.concat(all_tables)

# Create analyte group and units columns from analyte column using regex (analyte_group is in ALL CAPS and usually precedes units in parentheses; e.g. METALS (mg/kg)) - IT CANNOT HAVE A CONCENTRATION
clean_tables["analyte_group"] = clean_tables["analyte"].str.extract(r"^([A-Z\/]{2,}(?: [A-Z\/]+)*)(?:\s\((?:SU|[a-z]+\/[A-Za-z]+)\)$)?")
clean_tables.loc[clean_tables["conc_rl"] != "", "analyte_group"] = None
clean_tables["units"] = clean_tables["analyte"].str.extract(r"\((SU|%|[^\)]+\/[A-Za-z]+)?\)$")
clean_tables["units"] = clean_tables["units"].str.replace("µg/L", "ug/L")

# Group by table_number and fill down values for analyte group and units columns within each group
clean_tables["analyte_group"] = clean_tables.groupby("table_number")["analyte_group"].ffill()
clean_tables["units"] = clean_tables.groupby("table_number")["units"].ffill()

# Manually set units to "mg/kg" for tables "A-3", "A-4", "A-5", "A-6"
clean_tables.loc[clean_tables["table_number"].isin(["A-3", "A-4", "A-5", "A-6"]), "units"] = "mg/kg"

# Fill down missing locations
clean_tables["location_id"] = clean_tables.groupby("table_number")["location_id"].ffill()

# Filter out rows where conc_rl is ""
clean_tables = clean_tables[clean_tables["conc_rl"] != ""]

# Create column called "detected" which is False validator_qualifier contains "U" or conc_rl contains "<" or "ND" and True otherwise
clean_tables["detected"] = ~clean_tables["validator_qualifier"].str.contains("U", na=False) & ~clean_tables["conc_rl"].str.contains("<|ND|NC|R|--", na=True)

# Add column for significant figures in conc_rl
clean_tables["sigfigs"] = 2

# Remove non-numeric characters from conc_rl and convert conc_rl and mdl columns to numeric
clean_tables["conc_rl"] = clean_tables["conc_rl"].str.replace(r"[^0-9.]", "", regex=True)
clean_tables["conc_rl"] = pd.to_numeric(clean_tables["conc_rl"])
clean_tables["mdl"] = pd.to_numeric(clean_tables["mdl"])

# Remove all non-numeric characters from the level columns and convert all columns containing the text "level" or "limit" to numeric
level_columns = [col for col in clean_tables.columns if "level" in col or "limit" in col]
clean_tables[level_columns] = clean_tables[level_columns].replace(r"[^0-9.]", "", regex=True)
clean_tables[level_columns] = clean_tables[level_columns].apply(pd.to_numeric)

# Convert date_sampled to date (e.g. '29-Aug-2016' for table "A-9" and '8/20/2016' for all other tables)
clean_tables.loc[clean_tables["table_number"] == "A-9", "date_sampled"] = pd.to_datetime(clean_tables.loc[clean_tables["table_number"] == "A-9", "date_sampled"], format="%d-%b-%Y")
clean_tables.loc[clean_tables["table_number"] != "A-9", "date_sampled"] = pd.to_datetime(clean_tables.loc[clean_tables["table_number"] != "A-9", "date_sampled"], format="%m/%d/%Y")

# Extract table_order from table_number and page_order/result_order from index to preserve the order of the tables and results
clean_tables["table_order"] = clean_tables["table_number"].str.extract(r'(\d+)').astype(int)
clean_tables["page_order"] = clean_tables.index + 1
clean_tables["result_order"] = clean_tables.groupby("table_number").cumcount() + 1

# Reset the index
clean_tables = clean_tables.reset_index(drop=True)

# Re-order columns
meta_columns = ["table_order", "page_order", "result_order", "page_number", "table_number", "title", "subtitle", "site_name", "dataset"]
header_columns = ["location_id", "client_id", "lab_id", "date_sampled", "matrix", "depth_ft_bgs"]
clean_tables = clean_tables[meta_columns + header_columns + (clean_tables.columns.difference(meta_columns+header_columns+level_columns)).tolist() + level_columns]

# Save the clean tables to a pickle file and CSV using filename from pdf_path
fout = f"{output_dir}/{pdf_path.split('/')[-1].split('.')[0]}_clean"
clean_tables.to_pickle(f"{fout}.pkl")
clean_tables.to_csv(f"{fout}.csv", index=False)
clean_tables.to_excel(f"{fout}.xlsx", index=False)

In [36]:
# Read data from pickle file
clean_tables = pd.read_pickle(f"{fout}.pkl")

# Database Connection
engine = create_engine(os.getenv("DB_CONNECTION"))

# Write to PostgreSQL
clean_tables.to_sql(os.getenv("DB_TABLE"), engine, if_exists='replace', index=True)

# Close connection
engine.dispose()